In [6]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
assistant = client.beta.assistants.create(
    name = "Tutor de Tecnologia",
    instructions = "Você é um tutor sobre assuntos relacionados a tecnologia.",
    tools= [{"type": "code_interpreter"}],
    model = "gpt-3.5-turbo-0125"
)

In [11]:
pergunta = """"
Em um sistema de cache com mapeamento direto, dado uma memória principal 
de uma cache de 256 kb e blocos de 64 Bytes:
- Quantas linhas existem na cache?
- Onde o bloco de endreço 0x1A23C será mepado na cache?
"""

In [13]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta
)

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/3685932091.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/3685932091.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [14]:
# Executa a Trhead
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions= "Nome e usuário premium"
)

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/1703593232.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [16]:
run.status

'queued'

In [17]:
# Aguarda a Thread rodar
import time

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/3925348386.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [18]:
# Verificar a resposta

if run.status == "completed":
    mensagem = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagem)
else:
    print(f"Erro: {run.status}")    

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/1142915008.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagem = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_XAOsb3KRY5h1oG0KwMdByFF4', assistant_id='asst_DHyV86HP4wFbPF4zWGo7dQLH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Após a conversão do endereço hexadecimal 0x1A23C para binário, identificamos que:\n- Os bits do tag são '00'.\n- Os bits do índice da linha são '011010001000'.\n- Os bits do offset do bloco são '111100'.\n\nPortanto, o bloco de endereço 0x1A23C será mapeado na linha de índice '011010001000' da cache."), type='text')], created_at=1751348076, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_DOMxHRgSX7mc90BqEte5OB3M', status=None, thread_id='thread_uVS4kHwjHvBLbmlCaGhPWVkM'), Message(id='msg_3BKObvEf6ngpiOiguEQSKn2T', assistant_id='asst_DHyV86HP4wFbPF4zWGo7dQLH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Portanto, a cache com mapeamento direto terá 4096 linhas.\n\nA

In [19]:
print(mensagem.data[0].content[0].text.value)

Após a conversão do endereço hexadecimal 0x1A23C para binário, identificamos que:
- Os bits do tag são '00'.
- Os bits do índice da linha são '011010001000'.
- Os bits do offset do bloco são '111100'.

Portanto, o bloco de endereço 0x1A23C será mapeado na linha de índice '011010001000' da cache.


In [26]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/1597243276.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [27]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")

    if step.step_details.type == "toool_calls":
        for tool_call in step.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )

        print(message.content[0].text.value)



====Step message_creation


/var/folders/gq/3vbww1bj7x9dpmp6zy8llslw0000gn/T/ipykernel_43732/2949532723.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Para calcular o número de linhas na cache, precisamos dividir o tamanho total da cache pelo tamanho de cada bloco.

Dado:
- Tamanho da cache: 256 kB = 256 * 1024 bytes
- Tamanho do bloco: 64 bytes

Vamos calcular o número de linhas na cache.

====Step tool_calls

====Step message_creation
Portanto, a cache com mapeamento direto terá 4096 linhas.

Agora, para determinar onde o bloco de endereço 0x1A23C será mapeado na cache, podemos seguir os seguintes passos:
1. Primeiro, precisamos converter o endereço hexadecimal 0x1A23C para binário.
2. Em seguida, identificamos o tamanho do bloco em binário para encontrar os bits que representam o offset do bloco.
3. Por fim, determinamos a linha na cache onde o bloco será mapeado. 

Vamos realizar esses cálculos.

====Step tool_calls

====Step message_creation
Após a conversão do endereço hexadecimal 0x1A23C para binário, identificamos que:
- Os bits do tag são '00'.
- Os bits do índice da linha são '011010001000'.
- Os bits do offset do bloco são